In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai; fastai.__version__

In [ ]:
import pandas as pd
from fastai.tabular import *

In [ ]:
# the datasets
house_price_df = pd.read_csv("/content/drive/MyDrive/QUTREProject/Brisbane_suburbs_02_sales_2018_property_GB_Prop_dist16.csv", index_col=0)
house_price_df

In [ ]:
df_01 = house_price_df.loc[house_price_df['school_dist'] > 10000]
df_01

In [ ]:
house_price_df = house_price_df.drop(df_01.index)
house_price_df

In [ ]:
house_price_df.reset_index(drop=True, inplace=True)
house_price_df

In [ ]:
# test_df = house_price_df.loc[(house_price_df['suburb_id_x']=='QLD1346') | (house_price_df['suburb_id_x']=='QLD627') | (house_price_df['suburb_id_x']=='QLD24') | \
#                          (house_price_df['suburb_id_x']=='QLD3231') | (house_price_df['suburb_id_x']=='QLD32') | (house_price_df['suburb_id_x']=='QLD510') | \
#                          (house_price_df['suburb_id_x']=='QLD545') | (house_price_df['suburb_id_x']=='QLD718') | (house_price_df['suburb_id_x']=='QLD389') ]
# test_df

In [ ]:
train_df=house_price_df.sample(frac=0.8,random_state=200) #random state is a seed value
test_df=house_price_df.drop(train_df.index)
# train_df=house_price_df.drop(test_df.index)
train_df.reset_index()
test_df.reset_index()

In [ ]:
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
test_df.reset_index(drop=True, inplace=True)
test_df

In [ ]:
# Check the length of the dataset
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print(train_df.columns)

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum()

In [ ]:
# Let's check for the test set as well
test_df.isnull().sum()

In [ ]:
# Preprocessing
procs = [FillMissing, Categorify, Normalize]
#procs = [FillMissing, Normalize]

In [ ]:
# Split our variables into target, categorical and continuous variables
dep_var = 'price_value'

# cat_names = ['suburb_id_x']
# cat_names = ['street_id_x']
# cat_names = ['street_id_x', 'suburb_id_x']

cat_names = []
cont_names = ['propertyFeatures_bedrooms_x', 'propertyFeatures_bathrooms_x', 'propertyFeatures_carparks_x', 'landDetails_propertyArea_x', 'school_dist'\
              , 'hospital_dist', 'uni_dist', 'park_dist', 'mall_dist', 'CBD_dist', 'First_year_transaction', 'highway_dist', 'bus_stop_dist', \
       'ferry_terminal_dist', 'railway_station_dist', 'number_transaction', \
       'crime_rate_2017', 'state_school_percentage', \
       'class_size_achievement_pct_2017', 'Density', 'Score2017']

print(cat_names)
print(cont_names)

In [ ]:
test = TabularList.from_df(df=test_df, cat_names=cat_names, cont_names=cont_names, procs=procs)

In [ ]:
data = (TabularList.from_df(df=train_df, cat_names=cat_names, cont_names=cont_names, procs=procs)
                   .split_by_rand_pct()
                   .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
                   .add_test(test)
                   .databunch())

In [ ]:
data.show_batch(10)

In [ ]:
learn = tabular_learner(data, layers=[], metrics=[rmse,r2_score])

In [ ]:
#Show the complete Summary of the model
learn.summary

In [ ]:
learn.model_dir = '/content/drive/MyDrive/Fastai/'

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
min_grad_lr = learn.recorder.min_grad_lr
learn.fit_one_cycle(10, slice(min_grad_lr))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
#Display Predictions On Training Data
learn.show_results(ds_type=DatasetType.Train,rows = 5)
#Display Predictions On Validation Data
learn.show_results(ds_type=DatasetType.Valid)

In [ ]:
#Getting The Training And Validation Errors
tr = learn.validate(learn.data.train_dl)
va = learn.validate(learn.data.valid_dl)
print("The Metrics used In Evaluating The Network:", str(learn.metrics))
print("\nThe calculated RMSE & R-Squared For The Training Set :", tr[1:])
print("\nThe calculated RMSE & R-Squared For The Validation Set :", va[1:])

In [ ]:
#Plotting Momentum & Learning Rate
learn.recorder.plot_lr(show_moms=True)
#Plotting the metrics of evaluation
learn.recorder.plot_metrics()

In [ ]:
learn.model.embeds

In [ ]:
# variable = 'suburb_id_x'
# ix = cat_names.index(variable)

# var_vals = list(df[variable].astype('category').cat.categories.values)
# nval = len(var_vals)
# print(f'Number of values: {nval}')
# print(var_vals)

In [ ]:
# import altair as alt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# emb_mx = to_np(next(learn.model.embeds[ix].parameters()))
# # X_emb = TSNE(n_components=2, perplexity=3).fit_transform(emb_mx)
# X_emb = PCA(n_components=2).fit_transform(emb_mx)
# # annotation = np.append('Other', np.array(var_vals))
# annotation = np.array(var_vals)

In [ ]:
# print(annotation)

In [ ]:
# emb_df = pd.DataFrame(X_emb, columns=['Dim1', 'Dim2'])
# emb_df[variable]=annotation

In [ ]:
# points = alt.Chart(emb_df).mark_circle(size=60).encode(
#     x='Dim1',
#     y='Dim2',
#     tooltip=[variable]
# )

# text = points.mark_text(
#     align='left',
#     baseline='middle',
#     dx=7
# ).encode(
#     text=variable
# )

# points + text

In [ ]:
test_predictions = learn.get_preds(ds_type=DatasetType.Test)[0]

In [ ]:
import numpy as np

#Converting the tensor output to a list of predicted values
test_predictions = [np.exp(p[0].data.item()) for p in test_predictions]

# Create "submission.csv" file
submission = pd.DataFrame({'Id': test_df['id_combined'], 'price_value': test_df['price_value'], 'Predicted': test_predictions})
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
import pandas
import matplotlib.pyplot as plt
import statsmodels.api
import statsmodels.formula.api as sm
import scipy.stats as stats

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(2, 2, 1)
ax.scatter(submission['price_value'], submission['Predicted'])
ax.set_xlabel('value')
ax.set_ylabel('predicted')
ax.set_title(str(submission['Predicted'].corr(submission['price_value'])))

In [ ]:
import numpy as np
Y_true = submission['price_value']
pred = submission['Predicted']


#RMSLE
error = np.square(np.log(pred + 1) - np.log(Y_true +1)).mean() ** 0.5
print(error)

score = 1 - error
print("SCORE For test : ",score)


In [ ]:
dfCopy = test_df.copy()
dfCopy

In [ ]:
dfCopy['Predicted'] = test_predictions
dfCopy

In [ ]:
# dfCopy.to_csv('/content/drive/MyDrive/QUTREProject/2018_Hedonic_Pricing_Test_results_new2.csv')
# dfCopy.to_pickle('/content/drive/MyDrive/QUTREProject/2018_Hedonic_Pricing_Test_results_new2.pkl')